In [7]:
import pandas as pd
import requests
import json
import urllib.parse
from retrying import retry
import numpy as np
import time
import pymongo

In [8]:
# 获取每页json数据
@retry(stop_max_attempt_number=3)
def json_result(page, session, headers, data):
    data['pn'] = page
    if page == 1:
        data['first'] = 'true'
    res = session.post(url, headers=headers, data=data, timeout=(10, 10))
    if res.status_code != requests.codes.ok:
        print('page:', page, 'status_code:', res.status_code)
    res = res.json()
    return res

In [12]:
if __name__ == '__main__':
    # 常量配置
    name = '风控建模'
    referer = 'https://www.lagou.com/jobs/list_%s?labelWords=&fromSearch=true&suginput=' % urllib.parse.quote(name)
    url = 'https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
        'Referer': referer
    }
    data = {
        'first': 'false',
        'kd': name
    }
    
    # 连接mongo数据库
    client = pymongo.MongoClient('localhost', 27017)
    db = client['spider']
    db['lagou_%s' % name].create_index([('positionId', 1)], unique=True)  # 创建唯一索引
    
    # 维持会话
    s = requests.Session()
    s.get(referer, headers=headers)
    
    # 遍历每页
    pages = int(np.ceil(json_result(1, s, headers, data)['content']['positionResult']['totalCount'] / 15))
    time.sleep(5)
    print('总共%d页：' % pages)
    error = 0
    for page in range(1, pages):
        print('正在采集第%d页...' % page)
        results = json_result(page, s, headers, data)
        if results['content']['positionResult']['resultSize'] != 0:
            try:
                db['lagou_hr_info'].insert_many(list(results['content']['hrInfoMap'].values()))
                db['lagou_%s' % name].insert_many(results['content']['positionResult']['result'])
            except Exception as e:
                print(e)
                time.sleep(10)
                continue
        else:
            print('采集第%d页失败：' % page)
            time.sleep(10 + 10 * abs(np.random.randn()))
            error += 1
            if error >= 5:  # 连续失败五次则退出
                print('退出')
                break
        time.sleep(10)  # 缓解服务器压力

总共422页：
正在采集第1页...
正在采集第2页...
正在采集第3页...
正在采集第4页...
正在采集第5页...
正在采集第6页...
正在采集第7页...
正在采集第8页...
正在采集第9页...
正在采集第10页...
正在采集第11页...
正在采集第12页...
正在采集第13页...
正在采集第14页...
正在采集第15页...
正在采集第16页...
正在采集第17页...
正在采集第18页...
正在采集第19页...
正在采集第20页...
正在采集第21页...
正在采集第22页...
正在采集第23页...
正在采集第24页...
正在采集第25页...
正在采集第26页...
正在采集第27页...
正在采集第28页...
正在采集第29页...
正在采集第30页...
正在采集第31页...
正在采集第32页...
正在采集第33页...
正在采集第34页...
正在采集第35页...
正在采集第36页...
正在采集第37页...
正在采集第38页...
正在采集第39页...
正在采集第40页...
正在采集第41页...
正在采集第42页...
正在采集第43页...
正在采集第44页...
正在采集第45页...
正在采集第46页...
正在采集第47页...
正在采集第48页...
正在采集第49页...
正在采集第50页...
正在采集第51页...
正在采集第52页...
正在采集第53页...
正在采集第54页...
正在采集第55页...
正在采集第56页...
正在采集第57页...
正在采集第58页...
正在采集第59页...
正在采集第60页...
正在采集第61页...
正在采集第62页...
正在采集第63页...
正在采集第64页...
正在采集第65页...
正在采集第66页...
正在采集第67页...
正在采集第68页...
正在采集第69页...
正在采集第70页...
正在采集第71页...
正在采集第72页...
正在采集第73页...
正在采集第74页...
正在采集第75页...
正在采集第76页...
正在采集第77页...
正在采集第78页...
正在采集第79页...
正在采集第80页...
正在采集第81页...
正在采集第82页...
正在采集第83页...
正在采集第